In [1]:
%matplotlib widget

import os
import numpy as np
import pandas as pd
import itertools as it
from glob import glob
import matplotlib.pyplot as plt
from sbpy.calib import Sun
from astropy import units as u
from astropy.io import fits
from astropy.visualization import simple_norm
from copy import deepcopy as copy

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch

from lvmdrp.core.constants import LVM_UNAM_URL, CONFIG_PATH
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata, fix_lamps_metadata, get_masters_metadata

from lvmdrp.core.plot import plot_image
from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod, fluxCalMethod

plt.style.use("seaborn-v0_8-talk")

In [2]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.fits")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.fits")
out_arc_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{lamps}.fits")

In [3]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "lco_com"), ignore_cache=False).to_pandas()
frames_table = frames_table.loc[~frames_table.mjd.isin([60008,60009,60037,60038])]
# fix arc lamps metadata
fix_lamps_metadata(metadata=frames_table, lamp_names="argon neon ldls hgne xenon krypton".split())

[INFO]: loading cached metadata from '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/frames_table.pkl'


In [4]:
frames_table

,imagetyp,spec,mjd,camera,expnum,exptime,argon,neon,ldls,hgne,xenon,krypton,path
0,arc,sp1,60039,b1,00000527,90.0,False,False,False,False,True,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
1,object,sp1,60039,r1,00000502,900.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
2,arc,sp3,60039,b3,00000513,3.0,True,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
3,object,sp3,60039,b3,00000581,30.0,False,False,True,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
4,bias,sp3,60039,b3,00000539,0.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,object,sp1,60011,z1,00000222,5.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
2057,object,sp1,60011,z1,00000221,5.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
2058,bias,sp1,60011,r1,00000264,0.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
2059,arc,sp1,60011,z1,00000232,3.0,False,False,True,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...


In [5]:
in_bias = get_masters_metadata(path_pattern=out_calib_path, mjd=60011, kind="mbias").path
in_dark = get_masters_metadata(path_pattern=out_calib_path, mjd=60011, kind="mdark").path

mjd = "super"
kind = "xarc"
camera = "r1"
lamps = "neon"

in_rss = out_arc_path.format(mjd=mjd, kind=kind, camera=camera, lamps=lamps)

In [6]:
# write down some tophat passbands
wcent = [5500, 7200]
width = [500, 50]

rssMethod.DONE_PASS = []
for w, d in zip(wcent, width):
    wi, wf = w-d//2, w+d//2
    fname = os.path.join(CONFIG_PATH, f"{wi}-{wf}_passband.txt")
    
    rssMethod.DONE_PASS.append(f"{wi}-{wf}")
    
    wlength = np.arange(wi, wf+1, 1)
    passband = np.ones_like(wlength)
    passband[[0,-1]] = 0
    idx = np.arange(0, wlength.size, 1)
    
    passband_table = np.column_stack((idx, wlength, passband)).astype(int)
    np.savetxt(fname, passband_table, fmt=("%d", "%.2f", "%.2f"))

rssMethod.DONE_PASS

['5250-5750', '7175-7225']

In [7]:
rssMethod.quickQuality(
    in_sci=in_rss,
    in_std=in_rss,
    in_sky=in_rss,
    in_biases=in_bias,
    in_arc=in_rss,
    in_fiberflat=in_rss,
    out_report="test.fits",
    ref_values="../../python/lvmdrp/etc/quickdrp_refvals.yaml",
)

FileNotFoundError: [Errno 2] No such file or directory: './data/lvm-super-xarc-r1-neon.fits'